IBM Data Science Capstone course.
This notebook is for part 3 of week 3 assignment.
I copied the codes of the first 2 parts of this week's assignments and removed markdowns and comments. For the complete codes with markdowns, please see "capstone_w3_hw1" and "capstone_w3_hw2" in the same repository.

Part 1

In [18]:
#import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

# get wikipedia page html
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text
soup=BeautifulSoup(source,'lxml')

#after inspecting html of page, the class of the table is found
table = soup.find('table', {"class": "wikitable sortable"})

rows=table.find_all('tr') #all table's rows
# find name of columns from first row of table
col_hs=rows[0].find_all('th') #columns name have tag <th
col_names=[colh.text.rstrip('\n') for colh in col_hs]
table_data=[]
for row in rows[1:]:
    col_data=row.find_all('td') #table elements have tag <td
    col_data=[col.text.rstrip('\n') for col in col_data]
    table_data.append([cell_data for cell_data in col_data])
table_df = pd.DataFrame(table_data, columns=col_names) #create dataframe

# Remove rows that don't have an assigned borough
table_df1=table_df[table_df['Borough']!='Not assigned'].reset_index().drop(['index'],axis=1)

# Assign borough's name to neighborhoods that don't have a name
table_df1['Neighbourhood'][table_df1['Neighbourhood']=='Not assigned']=table_df1['Borough']

# list of unique post codes and corresponding boroughs
df2=table_df1.drop_duplicates(subset=['Postcode','Borough']).sort_values(by='Postcode',axis=0,ascending=True)
postc_list=df2['Postcode']
boro_list=df2['Borough']

neigh_list=[]
s=', ' #delimiter
for pc in postc_list:
    neigh_list.append(s.join(table_df1[table_df1['Postcode']==pc]['Neighbourhood']))

newdf=pd.DataFrame({'PostalCode':postc_list, 'Borough': boro_list,'Neighborhood': neigh_list}).reset_index(drop=True)
print('The new dataframe has {} rows'.format(newdf.shape[0]))

The new dataframe has 103 rows


Part 2 

In [19]:
# dataframe of coordinates
coord_df=pd.read_csv("Geospatial_Coordinates.csv")

newdf[['Latitude','Longitude']]=coord_df[['Latitude','Longitude']]
newdf.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


The third part of this week assignment starts here. Folium is used to visualize the neighborhoods in Toronto by using the dataframe created at the previous step. Of these neighborhoods, I chose to explore Downtown Toronto. Then I applied k-Means to cluster the venues found using Foursquare.

In [20]:
# import libraries 
from geopy.geocoders import Nominatim # to find latitude and longitude
import folium # for visualization

Geopy is used to get Toronto's coordinates and Folium to visualize its neighborhoods. 

In [21]:
address = 'Toronto, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# map of Toronto's neighbourhoods
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(newdf['Latitude'], newdf['Longitude'], newdf['Borough'], newdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map)  
toronto_map

c:\users\silvia\appdata\local\programs\python\python36\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


I chose to explore downtown Toronto, it's probably one of the first places I would visit. I used Geopy again for the coordinates and created a new database that only has neighborhoods in downtown Toronto. Then I used Folium again to visualize them.

In [22]:
# simplified dataframe of only boroughs with "Downtown Toronto" in their name
torDT=newdf[newdf['Borough']=="Downtown Toronto"].reset_index(drop=True)
address = 'Downtown Toronto, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
DT_lat = location.latitude
DT_lng = location.longitude
# map of Downtown Toronto
DT_toronto_map = folium.Map(location=[DT_lat, DT_lng], zoom_start=13)
# add markers to map
for lat, lng, label in zip(torDT['Latitude'], torDT['Longitude'], torDT['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(DT_toronto_map)  
DT_toronto_map

c:\users\silvia\appdata\local\programs\python\python36\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Next, I used Foursquare to explore downtown Toronto. I used the function to loop thorugh all the neighborhoods, print their names, the total number of venues and how many of them are unique.

In [23]:
# @hidden_cell
# credentials and version used to make calls to Foursquare API
CLIENT_ID = 'MJYGRGYWV3VGYP0NYNW1SAYGOOPQECONLMYVLJ3NEOLL0BFQ' 
CLIENT_SECRET = 'P4WNI3HTO02FXFKOBKKNPVTJFKWDLM21V13HG41QG5CW1IET' 
VERSION = '20180605' # Foursquare API version

In [27]:
# parameters used in calls to Foursquare
radius=500
LIMIT=100
# define the same function used in the lab to loop thorugh the neighborhoods and find the venues 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# call the function to find the venues in Downtown Toronto
DTtoronto_venues=getNearbyVenues(torDT['Neighborhood'], torDT['Latitude'], torDT['Longitude'])

print('{} venues were found in the {} neighborhoods of Downtown Toronto. {} of these venue are unique'.format(
    DTtoronto_venues.shape[0], 
    torDT.shape[0],
    len(DTtoronto_venues['Venue Category'].unique())))

DTtoronto_venues.head()

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
1283 venues were found in the 18 neighborhoods of Downtown Toronto. 204 of these venue are unique


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


I analyzed the neighborhoods to find the most common venues for each one of them. I decided to keep the latitudes and longitudes columns because when the rows are grouped by neighborhood, the order of the neighborhoods changes. Keeping the columns will be useful in the next step, to make sure that the clusers' label found with kMeans are properly associated to the corresponding neighborhood. In this step I also printed the sizes of the dataframes, to make sure they are correct.

In [59]:
# one hot encoding
DTtor_onehot = pd.get_dummies(DTtoronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
DTtor_onehot['Neighbourhood'] = DTtoronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [DTtor_onehot.columns[-1]] + list(DTtor_onehot.columns[:-1])
DTtor_onehot = DTtor_onehot[fixed_columns]
DTtor_onehot[['Neighbourhood Latitude','Neighbourhood Longitude']] = DTtoronto_venues[['Neighborhood Latitude','Neighborhood Longitude']]  
#confirm that the number of rows is same as number of venues and number of columns is same as number of unique venues+3
print('The number of venues is {}, the number of unique venues is {}.'.format(
    DTtor_onehot.shape[0],DTtor_onehot.shape[1])) 
DTtor_grouped = DTtor_onehot.groupby('Neighbourhood').mean().reset_index()
#confirm the number of rows is same as number of neighborhoods and number of columns is same as number of unique venues+3      
print('The number of neighborhoods is {}, the number of unique venues is {}.'.format(
    DTtor_grouped.shape[0],DTtor_grouped.shape[1])) 
DTtor_grouped

The number of venues is 1283, the number of unique venues is 207.
The number of neighborhoods is 18, the number of unique venues is 207.


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio,Neighbourhood Latitude,Neighbourhood Longitude
0,"Adelaide, King, Richmond",0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00000,0.010000,0.000000,0.000000,0.010000,0.01,0.000000,43.650571,-79.384568
1,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,43.644771,-79.373306
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,43.628947,-79.394420
3,"Cabbagetown, St. James Town",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.020833,43.667967,-79.367675
4,Central Bay Street,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00000,0.012195,0.000000,0.000000,0.012195,0.00,0.012195,43.657952,-79.387383
5,"Chinatown, Grange Park, Kensington Market",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00000,0.060000,0.000000,0.040000,0.010000,0.00,0.000000,43.653206,-79.400049
6,Christie,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,43.669542,-79.422564
7,Church and Wellesley,0.00,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00000,0.011364,0.011364,0.011364,0.000000,0.00,0.011364,43.665860,-79.383160
8,"Commerce Court, Victoria Hotel",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,43.648199,-79.379817
9,"Design Exchange, Toronto Dominion Centre",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.01000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,43.647177,-79.381576


Next I created the dataframe with the 10 most common venues for each neighborhood. 

In [95]:
import numpy as np
# Find 10 most common venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues=10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe with no latitude, longitude columns
DTtor_common=DTtor_grouped.drop(['Neighbourhood Latitude','Neighbourhood Longitude'], axis= 1)
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DTtor_common['Neighbourhood']

for ind in np.arange(DTtor_common.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DTtor_common.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Cosmetics Shop,Gym,Restaurant,Hotel,Bar
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Cheese Shop,Café,Steakhouse,Beer Bar,Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Plane,Boat or Ferry,Airport Gate,Airport
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Pub,Bakery,Pizza Place,Café,Chinese Restaurant,Indian Restaurant,Italian Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Bar,Burger Joint,Japanese Restaurant,Spa,Salad Place
5,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Noodle House
6,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Nightclub,Convenience Store,Restaurant,Baby Store
7,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Burger Joint,Sushi Restaurant,Restaurant,Mediterranean Restaurant,Pub,Men's Store,Café
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Gastropub,Steakhouse,Italian Restaurant,Gym
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Hotel,Café,American Restaurant,Sports Bar,Italian Restaurant,Deli / Bodega,Gastropub,Gym,Restaurant


By looking at the dataframe of the most common venues in downtown Toronto, it can be noticed that:
- there are food venues in most neighborhoods
- Rosedale is an outdoor activities neighborhood
- as expected, there are air transportation related venues where the airport is located.
K-means can help get a better insight and find patterns in the venues' location by clustering the neighborhoods. So I ran the algorithm, using 5 clusters. 

In [97]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5
DTtor_grouped_clustering = DTtor_grouped.drop(['Neighbourhood', 'Neighbourhood Latitude','Neighbourhood Longitude'], axis= 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DTtor_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_

# add cluster's labels 
clustersdf=neighborhoods_venues_sorted
clustersdf['Cluster label'] = kmeans.labels_
clustersdf


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster label
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Cosmetics Shop,Gym,Restaurant,Hotel,Bar,0
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Cheese Shop,Café,Steakhouse,Beer Bar,Restaurant,0
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Plane,Boat or Ferry,Airport Gate,Airport,2
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Pub,Bakery,Pizza Place,Café,Chinese Restaurant,Indian Restaurant,Italian Restaurant,0
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Bar,Burger Joint,Japanese Restaurant,Spa,Salad Place,0
5,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Noodle House,3
6,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Nightclub,Convenience Store,Restaurant,Baby Store,4
7,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Burger Joint,Sushi Restaurant,Restaurant,Mediterranean Restaurant,Pub,Men's Store,Café,0
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Gastropub,Steakhouse,Italian Restaurant,Gym,0
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Hotel,Café,American Restaurant,Sports Bar,Italian Restaurant,Deli / Bodega,Gastropub,Gym,Restaurant,0


By looking at the cluster labels, the initial observation regarding Rosedale and the airport neighborhood are confirmed. Since they have unique venues, their label is only assigned to them. Separate dataframes can be created for each cluster to better look at their discriminating categories.

In [100]:
cl1=clustersdf.loc[clustersdf['Cluster label'] == 0, clustersdf.columns[0:clustersdf.shape[1]-1]]
cl1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Cosmetics Shop,Gym,Restaurant,Hotel,Bar
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Cheese Shop,Café,Steakhouse,Beer Bar,Restaurant
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Pub,Bakery,Pizza Place,Café,Chinese Restaurant,Indian Restaurant,Italian Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Bar,Burger Joint,Japanese Restaurant,Spa,Salad Place
7,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Burger Joint,Sushi Restaurant,Restaurant,Mediterranean Restaurant,Pub,Men's Store,Café
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Gastropub,Steakhouse,Italian Restaurant,Gym
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Hotel,Café,American Restaurant,Sports Bar,Italian Restaurant,Deli / Bodega,Gastropub,Gym,Restaurant
10,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Steakhouse,American Restaurant,Deli / Bodega,Bar,Gym,Gastropub
12,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Hotel,Aquarium,Pizza Place,Café,Italian Restaurant,Scenic Lookout,Sports Bar,Brewery,Fried Chicken Joint
13,"Harbourfront, Regent Park",Coffee Shop,Bakery,Café,Park,Mexican Restaurant,Restaurant,Breakfast Spot,Pub,Theater,Yoga Studio


In [101]:
cl2=clustersdf.loc[clustersdf['Cluster label'] == 1, clustersdf.columns[0:clustersdf.shape[1]-1]]
cl2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,Park,Playground,Trail,Yoga Studio,Department Store,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [102]:
cl3=clustersdf.loc[clustersdf['Cluster label'] == 2, clustersdf.columns[0:clustersdf.shape[1]-1]]
cl3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Plane,Boat or Ferry,Airport Gate,Airport


In [103]:
cl4=clustersdf.loc[clustersdf['Cluster label'] == 3, clustersdf.columns[0:clustersdf.shape[1]-1]]
cl4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Noodle House
11,"Harbord, University of Toronto",Café,Theater,Bakery,Bar,Bookstore,Restaurant,Japanese Restaurant,Coffee Shop,Chinese Restaurant,Poutine Place


In [104]:
cl5=clustersdf.loc[clustersdf['Cluster label'] == 4, clustersdf.columns[0:clustersdf.shape[1]-1]]
cl5

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Nightclub,Convenience Store,Restaurant,Baby Store


Finally, the clusters can be visualized on a map. I preferred to use a new,clean dataframe.

In [115]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_cols=['Neighborhood','Latitude','Longitude','Cluster']
mapdf=pd.DataFrame(columns=map_cols)
mapdf[['Neighborhood','Latitude','Longitude']]=DTtor_grouped[['Neighbourhood', 'Neighbourhood Latitude','Neighbourhood Longitude']]
mapdf['Cluster']=clustersdf['Cluster label']
#mapdf
# create map
map_clusters = folium.Map(location=[DT_lat, DT_lng], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mapdf['Latitude'], mapdf['Longitude'], mapdf['Neighborhood'], mapdf['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


After looking at k_Means results and Folium map, I decided to give the following names to the clusters:
cluster_label 0: "downtown food"
cluster_label 1: "fun in the park"
cluster_label 2: "airport"
cluster_label 3: "more downtown food"
cluster_label 4: "all you need is here!"